In [ ]:
#En la primera ejecución, descargamos e instalamos la librería Spacy y el vocabulario español de tamaño grande
#!pip install -U spacy
#!python -m spacy download es_core_news_lg

In [3]:
#Se importan las librerías necesaarias
import spacy
import json
import pickle
import numpy as np
import random
import re

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [4]:
#Leemos el fichero intents mediante la librería json para cargar los datos y cargamos el vocabulario
file = open('intents.json',encoding = 'utf-8').read()
intents = json.loads(file)
nlp = spacy.load(r'C:\Users\fdome\AppData\Local\Programs\Python\Python39\Lib\site-packages\es_core_news_lg\es_core_news_lg-3.4.0')


In [5]:
#Generamos las estructuras que vamos a utilizar a lo largo del modelo
words=[]
lemmas = []
etiquetas = []
pares = []
stops = ['a','yo','tu','el', 'nosotros', 'vosotros', 'ellos', 'me', 'mi', 'te', 'ti', 'la', 'los', 'las', 'y', 'o', 'al','del','ya','que']

In [6]:
#Leemos los datos que hemos guardado del archivo de intenciones, y para cada uno de los patrones o frases
#Primero eliminamos carácteres no deseados, pasamos a minúsculas y quitamos acentos
#Aplicamos el método nlp de spacy para poder obtener los lemas de cada una de las palabras del patrón
#Almacenamos en pares el conjunto de lemas de cada patron con su correspondiente etiqueta
for intent in intents['intents']:
    for patron in intent['patrones']:
        
        w=re.sub(r'[¿¡]', "", patron)
        w=re.sub(r'\W+', " ", w)
        w=w.lower()
        w=re.sub('á', 'a', w)
        w=re.sub('é', 'e', w)
        w=re.sub('í', 'i', w)
        w=re.sub('ó', 'o', w)
        w=re.sub(r'[üú]', 'u', w)
        doc = nlp(w)
        words=[]
        for token in doc:
            if token.text not in stops:
                words.append(token.lemma_)
                lemmas.append(token.lemma_)  
        pares.append((words,intent['etiqueta']))
        if intent['etiqueta'] not in etiquetas:
            etiquetas.append(intent['etiqueta'])


In [7]:
#Creamos sets de los lemmas y las etiquetas para tener datos únicos ordenados y los guardamos para usuarlos en el bot
lemmas=sorted(list(set(lemmas)))
etiquetas=sorted(list(set(etiquetas)))
print(len(pares),'pares')
print(len(etiquetas), "etiquetas",etiquetas)
print(len(lemmas), "lemas unicos",lemmas)

pickle.dump(lemmas,open('lemmas.pkl','wb'))
pickle.dump(etiquetas,open('etiquetas.pkl','wb'))

450 pares
34 etiquetas ['acciones', 'actividades', 'adios', 'amor', 'arte', 'bebida', 'chiste', 'comida', 'conexion', 'conocer', 'edad', 'estaciones', 'estado', 'familia', 'físico', 'gracias', 'insulto', 'jugar', 'mascotas', 'medico', 'moda', 'motivacion', 'musica', 'negatividad', 'nombre', 'opciones', 'pelicula', 'positividad', 'repetir', 'risa', 'salud', 'saludos', 'tiempo', 'vacaciones']
560 lemas unicos ['abrigar yo', 'abstracto', 'abuela', 'aburrir', 'acabar', 'acuario', 'adio', 'age', 'agobiado', 'agobiar', 'agua', 'ahi', 'ahora', 'aire', 'ajustado', 'alcohol', 'alegrame', 'alegre', 'alegrir', 'algo', 'alguien', 'algun', 'alguno', 'alien', 'alto', 'amigo', 'and', 'animado', 'animal', 'animame', 'animar', 'antes', 'apagar', 'aqui', 'armario', 'arreglar', 'arte', 'artista', 'asiatico', 'atareado', 'ayer', 'ayuda', 'ayudar', 'ayudar yo', 'azul', 'año', 'bajito', 'bajo', 'balada', 'bar', 'barrer', 'barriga', 'bastante', 'basura', 'bata', 'baño', 'beber', 'bien', 'blanco', 'buen', 'bu

In [8]:
#Creamos la bolsa de palabras de cada una de las frases de los patrones almacenados en la estructura pares
#Además codificamos la salida a one hot encoding para su uso con la red neuronal
#finalmente dividimos el conjunto en parte de entrenamiento y test para las comprobaciones
training=[]
salida_vacia = [0]*len(etiquetas)
for par in pares:
    bow = []
    patron = par[0]
    for lemma in lemmas:
        bow.append(1) if lemma in patron else bow.append(0)
    salida = list(salida_vacia)
    salida[etiquetas.index(par[1])] = 1
    training.append([bow,salida])

training=np.array(training,dtype=object)
train,test = train_test_split(training, test_size=0.2,random_state=360)
train_X = list(train[:,0])
train_y = list(train[:,1])
test_X = list(test[:,0])
test_y = list(test[:,1])

In [14]:
#Generamos el modelo de la red neuronal con tres capas y dos de dropout para evitar sobreentrenamiento
#Lo compilamos, lo entrenamos con el conjunto de entrenamiento y guardamos el modelo final
model = Sequential()
model.add(Dense(512,input_shape=(len(train_X[0]),),activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(len(train_y[0]),activation='softmax'))

opt=Adam(lr=0.0001, decay=1e-6)
model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
mod=model.fit(train_X,train_y, epochs=100, batch_size=3, verbose=1)
model.save('jackBot_model.h5',mod)


Epoch 1/100
120/120 [==============================] - 0s 3ms/step - loss: 3.5173 - accuracy: 0.0389
Epoch 2/100
120/120 [==============================] - 0s 3ms/step - loss: 3.4914 - accuracy: 0.0500
Epoch 3/100
120/120 [==============================] - 0s 3ms/step - loss: 3.4566 - accuracy: 0.1083
Epoch 4/100
120/120 [==============================] - 0s 3ms/step - loss: 3.4183 - accuracy: 0.1167
Epoch 5/100
120/120 [==============================] - 0s 3ms/step - loss: 3.3653 - accuracy: 0.1250
Epoch 6/100
120/120 [==============================] - 0s 3ms/step - loss: 3.3325 - accuracy: 0.1417
Epoch 7/100
120/120 [==============================] - 0s 3ms/step - loss: 3.2741 - accuracy: 0.1611
Epoch 8/100
120/120 [==============================] - 0s 3ms/step - loss: 3.2137 - accuracy: 0.1556
Epoch 9/100
120/120 [==============================] - 0s 3ms/step - loss: 3.1621 - accuracy: 0.2278
Epoch 10/100
120/120 [==============================] - 0s 3ms/step - loss: 3.1084 - accura

In [16]:
#evaluamos el modelo con los datos de test
scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

3/3 [==============================] - 0s 2ms/step - loss: 1.7732 - accuracy: 0.5778

accuracy: 57.78%


In [22]:
#Evaluamos el modelo con los datos de entrenamiento
scores = model.evaluate(train_X, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.save('jackBot_model.h5',mod)

12/12 [==============================] - 0s 2ms/step - loss: 0.0163 - accuracy: 0.9972

accuracy: 99.72%
